# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察均值編碼的效果

# [作業重點]
- 仿造範例, 完成標籤編碼與均值編碼搭配邏輯斯迴歸的預測
- 觀察標籤編碼與均值編碼在特徵數量 / 邏輯斯迴歸分數 / 邏輯斯迴歸時間上, 分別有什麼影響 (In[3], Out[3], In[4], Out[4]) 

# 作業1
* 請仿照範例，將鐵達尼範例中的類別型特徵改用均值編碼實作一次

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

# 讀取訓練與測試資料
# 讀取 csv 資料檔, 並觀察前幾筆資料 ( .head() )
data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

# 訓練資料需要 train_X, train_Y / 預測輸出需要 ids(識別每個預測值), test_X
# 在此先抽離出 train_Y 與 ids, 而先將 train_X, test_X 該有的資料合併成 df, 先作特徵工程
train_Y = df_train['Survived']
ids = df_test['PassengerId']
# 先抽離出 train_Y 與 id
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
# 將train_X, test_X 該有的資料合併成 df, 先作特徵工程
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
# 只取類別值 (object) 型欄位, 存於 object_features 中
# zip(): 它接受一系列可迭代的對象作為參數，將對象中對應的元素打包成一個個 tuple（元組），然後返回由這些 tuples 組成的 list（列表）。若傳入參數的長度不等，則返回list的長度和參數中長度最短的對象相同。
# df.fillna():使用指定的方法填充NA / NaN值

object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Numeric Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
# 用'None'取代nan
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
* 觀察鐵達尼生存預測中，均值編碼與標籤編碼兩者比較，哪一個效果比較好? 可能的原因是什麼?

# 標籤編碼 + 邏輯斯迴歸

In [3]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    # Label encoding : 把每個類別 mapping 到某個整數，不會增加新欄位
    # fit來學習編碼，transform進行編碼
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]

# 設定模型與模型參數
# LogisticRegression(C =):逆正規化強度，較小的值指定更強的正則化。為正則化係數λ 的倒數，通常默認為1。
# 分類器(estimator)設定為邏輯斯迴歸
estimator = LogisticRegression()
# 設定程式開始計算時間
start = time.time() # 傳回從 1970/1/1 00:00:00 算起至今的秒數
print(f'shape : {train_X.shape}')
# 進行交叉焓驗證，並計算交叉驗證的平均值
print(f'score : {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}') # cv參數用於規定將原始數據分成多少份|
print(f'time : {time.time() - start} sec')

shape : (891, 5)
score : 0.7800138095537004
time : 0.1908872127532959 sec


# 均值編碼 + 邏輯斯迴歸

In [4]:
# 均值編碼 + 邏輯斯迴歸
data = pd.concat([df[:train_num], train_Y], axis=1)
for c in df.columns:
    # groupby():將資料依照自己要的column分組
    # reset_index()可以讓index重置成原本的樣子
    mean_df = data.groupby([c])['Survived'].mean().reset_index()
    mean_df.columns = [c, f'{c}_mean']
    # pd.merge():不同資料集依照某些欄位（屬性）進行合併操作
    data = pd.merge(data, mean_df, on=c, how='left')
    data = data.drop([c] , axis=1)
data = data.drop(['Survived'] , axis=1)

# 設定模型與模型參數
# LogisticRegression(C =):逆正規化強度，較小的值指定更強的正則化。為正則化係數λ 的倒數，通常默認為1。
# 分類器(estimator)設定為邏輯斯迴歸
estimator = LogisticRegression()
# 設定程式開始計算時間
start = time.time() # 傳回從 1970/1/1 00:00:00 算起至今的秒數
print(f'shape : {train_X.shape}')
# 進行交叉焓驗證，並計算交叉驗證的平均值
print(f'score : {cross_val_score(estimator, data, train_Y, cv=5).mean()}') # cv參數用於規定將原始數據分成多少份|
print(f'time : {time.time() - start} sec')

shape : (891, 5)
score : 1.0
time : 0.044972896575927734 sec


均值編碼比較好，但這主要是因為'Name'與'Ticket'兩個欄位的種類數眾多
如果均值編碼時加入這裡個欄位的均值會過擬合( overfitting )
因此下列程式碼中，需要先移除這兩個欄位，再做均值編碼

In [5]:
# 如果欄位類別數太多, 需要先剔除, 否則會 overfit (正確率會到達 1.0)
train_X.nunique()

Name        891
Sex           2
Ticket      681
Cabin       148
Embarked      4
dtype: int64

In [6]:
# 均值編碼 + 邏輯斯迴歸
data = pd.concat([df[:train_num], train_Y], axis=1)
for c in df.columns:
    # groupby():將資料依照自己要的column分組
    # reset_index()可以讓index重置成原本的樣子
    mean_df = data.groupby([c])['Survived'].mean().reset_index()
    mean_df.columns = [c, f'{c}_mean']
    # pd.merge():不同資料集依照某些欄位（屬性）進行合併操作
    data = pd.merge(data, mean_df, on=c, how='left')
    data = data.drop([c] , axis=1)
data = data.drop(['Survived', 'Name_mean', 'Ticket_mean'] , axis=1)
# 分類器(estimator)設定為邏輯斯迴歸
estimator = LogisticRegression()
# 設定程式開始計算時間
start = time.time() # 傳回從 1970/1/1 00:00:00 算起至今的秒數
print(f'shape : {train_X.shape}')
# 進行交叉焓驗證，並計算交叉驗證的平均值
print(f'score : {cross_val_score(estimator, data, train_Y, cv=5).mean()}') # cv參數用於規定將原始數據分成多少份|
print(f'time : {time.time() - start} sec')

shape : (891, 5)
score : 0.835019772770071
time : 0.05596470832824707 sec
